In [57]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from IPython.display import display

In [59]:
driver = webdriver.Chrome()

In [60]:
api_key = "RGAPI-ec132e4d-77ce-4de1-bb45-fdaf011ba5b6"

In [61]:
leagueacc_url="https://porofessor.gg/live/na/Cytosine-NA1"

In [62]:
driver.get(leagueacc_url)
ally_url=[]
ally_kda_list=[]
enemy_kda_list=[]

In [66]:
ally_tag=list(player.find_element(By.XPATH, '..').get_attribute('data-summonername') for player in driver.find_elements(By.XPATH, "//div[contains(@class, 'card card-5')]/div[contains(@class, 'cardHeader blue')]"))

In [67]:
enemy_tag=list(player.find_element(By.XPATH, '..').get_attribute('data-summonername') for player in driver.find_elements(By.XPATH, "//div[contains(@class, 'card card-5')]/div[contains(@class, 'cardHeader red')]"))

In [120]:
for player in ally_tag:
    print(player)
    name, tag = player.split("#")
    puuid_api_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}" + '?api_key=' + api_key
    player_info_raw = requests.get(puuid_api_url)
    while player_info_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        player_info_raw = requests.get(puuid_api_url)
    player_info = player_info_raw.json()['puuid']

    matches_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{player_info}/ids?start=0&count=20" + '&api_key=' + api_key
    matches_info_raw = requests.get(matches_api_url)
    while matches_info_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        matches_info_raw = requests.get(matches_api_url)
    matches_info= list(matches_info_raw.json())
        
    for match in matches_info:
        match_api_url= f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}" + '?api_key=' + api_key
        match_info_raw = requests.get(match_api_url)
        while match_info_raw.status_code == 429:
            print("Sleeping")
            time.sleep(10)
            match_info_raw = requests.get(match_api_url)
        match_info= match_info_raw.json()
        participantIndex = match_info['metadata']['participants'].index(player_info)
        matchData=match_info['info']['participants'][participantIndex]
        if (match_info['info']['queueId']==420) and (matchData['timePlayed']>180):
            solo_testing_key_stats = ['championName','teamPosition','kills', 'deaths', 'assists', 'damageDealtToBuildings', 'deaths', 'visionScore','doubleKills', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'goldEarned', 'inhibitorKills', 'inhibitorTakedowns', 'killingSprees', 'objectivesStolen', 'totalMinionsKilled', 'totalDamageDealt', 'turretKills','win']
            team_testing_key_stats = ['kda', 'damagePerMinute', 'goldPerMinute', 'teamDamagePercentage','dodgeSkillShotsSmallWindow', 'takedowns']
            support_solo_key_stats = ['detectorWardsPlaced']
            support_team_key_stats = ['controlWardsPlaced', 'completeSupportQuestInTime']
            jungle_solo_key_stats = ['neutralMinionsKilled', 'detectorWardsPlaced', 'dragonKills']
            jungle_team_key_stats = ['jungleCsBefore10Minutes']
            base_stats = {
                    **{stat: matchData[stat] for stat in solo_testing_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in team_testing_key_stats}
                }
            if matchData['teamPosition']== 'UTILITY':
                role_stats={
                    **{stat: matchData[stat] for stat in support_solo_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in support_team_key_stats}
                }
            elif matchData['teamPosition']== 'JUNGLE':
               role_stats={
                    **{stat: matchData[stat] for stat in jungle_solo_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in jungle_team_key_stats}
                }
            else:
                role_stats={}
            timePlayed = f"{matchData['timePlayed'] // 60}:{matchData['timePlayed'] % 60:02d}"
            df=pd.DataFrame([{**base_stats, **role_stats}])
            df.insert(0, 'timePlayed', timePlayed)
            df.index = ['']
            pd.set_option('display.max_columns', None)  
            pd.set_option('display.expand_frame_repr', False)
            # pd.set_option('display.width', 1000)
            display(df)
            # print(match_info['info']['participants'][participantIndex], end =" ")
            # print(match_info['info']['participants'][participantIndex]['kills'], end =" ")
            # print(match_info['info']['participants'][participantIndex]['deaths'], end =" ")
            # print(match_info['info']['participants'][participantIndex]['assists'])

Pineapple#Oreo


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,29:11,Riven,TOP,8,2,3,8808,14,0,False,True,False,True,13668,0,0,2,0,260,192587,2,False,5.5,864.210637,468.207511,0.288827,0,11


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,37:16,Riven,TOP,5,8,5,657,20,0,False,False,False,False,12553,0,0,2,0,192,157120,0,False,1.25,411.369117,336.696042,0.100456,4,10


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,31:36,Riven,TOP,3,11,0,3206,7,0,False,False,False,False,11445,0,0,0,0,221,179230,1,False,0.272727,757.478547,362.014872,0.262815,1,3


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,36:00,Riven,TOP,5,11,4,11058,17,1,False,False,False,False,15149,0,0,2,0,247,208107,4,False,0.818182,810.612658,420.669062,0.188654,2,9


...

,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,detectorWardsPlaced,controlWardsPlaced,completeSupportQuestInTime
,26:44,Janna,UTILITY,1,7,7,0,44,0,False,False,False,False,6135,0,0,0,0,20,13533,0,False,1.142857,160.279021,228.765167,0.06731,3,8,0,0,1


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,detectorWardsPlaced,controlWardsPlaced,completeSupportQuestInTime
,26:38,Janna,UTILITY,0,2,6,0,72,0,False,False,False,False,5916,0,0,0,0,34,15394,0,False,3,200.835947,222.082216,0.068007,1,6,3,3,0


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,detectorWardsPlaced,controlWardsPlaced,completeSupportQuestInTime
,34:25,Soraka,UTILITY,0,11,18,0,67,0,False,False,False,False,8089,0,0,0,0,26,21434,0,False,1.636364,202.263221,234.948732,0.057587,0,18,2,2,0


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,detectorWardsPlaced,controlWardsPlaced,completeSupportQuestInTime
,33:45,Janna,UTILITY,0,6,15,1999,90,0,False,False,False,False,9300,0,3,0,0,31,25108,2,True,2.5,177.529893,275.415323,0.050497,0,15,4,4,0


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,neutralMinionsKilled,detectorWardsPlaced,dragonKills,jungleCsBefore10Minutes
,34:00,Ivern,JUNGLE,3,9,15,0,29,0,False,False,False,False,11090,0,0,0,0,24,187110,0,False,2,567.265397,326.13593,0.148036,5,18,126,3,1,50.5


In [ ]:
requests.get(api)

In [ ]:
# for player in ally_tag:
#     print(player)
#     name, tag = player.split("#")
#     puuid_api_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}" + '?api_key=' + api_key
#     # player_info_raw = requests.get(puuid_api_url)
#     player_info = requests.get(puuid_api_url).json()['puuid']
#     matches_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{player_info}/ids?start=0&count=20" + '&api_key=' + api_key
#     # matches_info_raw = requests.get(matches_api_url)
#     matches_info = list(requests.get(matches_api_url).json())
#     for match in matches_info:
#         match_api_url= f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}" + '?api_key=' + api_key
#         # match_info_raw = requests.get(match_api_url)
#         match_info =requests.get(match_api_url).json()
#         participantIndex = match_info['metadata']['participants'].index(player_info)
#         print(match_info['info']['participants'][participantIndex]['kills'], end =" ")
#         print(match_info['info']['participants'][participantIndex]['deaths'], end =" ")
#         print(match_info['info']['participants'][participantIndex]['assists'])

In [ ]:
for player in ally_url:
    print(player)

In [429]:
# for player in ally_tag:
#     name, tag = player.split("#")
#     name = name.replace(" ", "%20")
#     tag = tag.replace(" ", "%20")
#     ally_url.append(f"https://www.op.gg/summoners/na/{name}-{tag}")
#     print(f"https://www.op.gg/summoners/na/{name}-{tag}")
# for player in enemy_tag:
#     name, tag = player.split("#")
#     name = name.replace(" ", "%20")
#     tag = tag.replace(" ", "%20")
#     ally_url.append(f"https://www.op.gg/summoners/na/{name}-{tag}")
#     print(f"https://www.op.gg/summoners/na/{name}-{tag}")
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------
# for player in ally_url:
#     driver.get(player)
#     driver.get(ally_url[0])
#     kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
#     deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
#     assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
#     kda=(kills+assists)/deaths
#     ally_kda_list.append(kda)
#     print(kda)kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
# deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
# assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
# kda=(kills+assists)/deaths
# ally_kda_list.append(kda)
# print(kda)
# for player in enemy_url:
#     driver.get(player)
#     kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
#     deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
#     assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
#     kda=(kills+assists)/deaths
#     enemy_kda_list.append(kda)
#     print(kda)
# ally_average_kda=sum(ally_kda_list)/5
# enemy_average_kda=sum(enemy_kda_list)/5